<a href="https://colab.research.google.com/github/dnguyend/StiefelGeodesic/blob/main/colab/FlagLogarithmV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TESTING THE LOGARITHM MAP FOR FLAG MANIFOLDS

## Usage note: User can just view results in the notebook or can rerun on the colab environment. Colab hides some cells if the workbook is long, just click on the hidden group, the hidden cells will expand.

## eventually we will fold this repository to ManNullRange, but since we are trying to freeze development there for now, we will import the code we need from that project.

## For Flag manifolds, the main algorithms are in two functions log_lbfgs and log_steep_descent. To view the code the user can go to the github page, or simply type ??log_lbfgs, for example, in a new empty cell. There is also log_descent (ie without the line search - it works less well)



In [1]:
!git clone  https://github.com/pymanopt/pymanopt.git
import sys
sys.path.append("/content/pymanopt")


Cloning into 'pymanopt'...
remote: Enumerating objects: 4127, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4127 (delta 0), reused 0 (delta 0), pack-reused 4124
Receiving objects: 100% (4127/4127), 899.14 KiB | 16.35 MiB/s, done.
Resolving deltas: 100% (2876/2876), done.


In [2]:
!git clone https://github.com/dnguyend/ManNullRange.git

Cloning into 'ManNullRange'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 253 (delta 164), reused 106 (delta 60), pack-reused 0
Receiving objects: 100% (253/253), 770.12 KiB | 7.86 MiB/s, done.
Resolving deltas: 100% (164/164), done.


In [3]:
!git clone https://github.com/dnguyend/StiefelGeodesic.git

Cloning into 'StiefelGeodesic'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 28 (delta 6), reused 16 (delta 4), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [4]:
from StiefelGeodesic.manifolds.LogFlag import log_lbfgs, log_descent, log_steep_descent
from StiefelGeodesic.manifolds.tools import linf, sbmat

In [5]:
import numpy as np
from numpy.random import (randint, randn)
from numpy import zeros, zeros_like, trace, allclose

from ManNullRange.manifolds.RealFlag import RealFlag
from ManNullRange.tests.test_tools import (
    check_zero, make_sym_pos, random_orthogonal, timeout,
    TimeoutError)



def compare_with_grassmann(n, d, NN):
    import numpy.linalg as la
    from numpy import bmat
    from scipy.linalg import expm, expm_frechet
    from ManNullRange.manifolds.tools import (
        vecah, unvecah, asym)
    from pymanopt.manifolds import Grassmann
    
    np.random.seed(0)    
    alp = np.zeros(2)
    alp[0] = 1
    alp[1] = .8

    dvec = np.array([n-d, d])
    q = dvec.shape[0]-1
    alpha = randint(1, 10, (q, q+1)) * .1
    alpha[:, 0] = alp[0]
    alpha[:, 1:] = alp[1]
    m, d = dvec.sum(), dvec[1:].sum()
    man = RealFlag(
        dvec, alpha=alpha,
        log_stats=True, log_method='trust-krylov')

    gr = Grassmann(m, d)
    
    X = gr.rand()
    # xi = gr.randvec(X)
    # X1 = gr.exp(X, np.pi*xi)

    cnt = 0
    # NN = 1000
    for i in range(NN):
        xi = gr.randvec(X)
        X1 = gr.exp(X, 0.6*np.pi*xi)
        # X1 = gr.rand()
        v1 = gr.dist(X, X1)
        # lgs, sts = man.log(X, X1, init_type=0)
        lgs, sts = log_lbfgs(man, X, X1)
        v2 = man.norm(X, lgs)
        if np.abs(v1 - v2) > 1e-4:
            print(i, v1, v2, v1-v2, (v1+v2)/np.pi)
            print(check_zero(X.T@lgs))
            X1a = man.exp(X, lgs)
            print(check_zero(X1a.T@X1a - np.eye(X1a.shape[1])))
            print(sts)
            # pdb.set_trace()
            cnt += 1
    print('Number of discrepency %d, percent discrepency %f' % (cnt, cnt/NN))
    
    print('Check that the aligment matrix is orthogonal')
    X1a = man.exp(X, lgs)
    F = X1.T@X1a
    print('L1 norm of diff is %f' % np.max(np.abs(F.T@F - np.eye(d))))

compare_with_grassmann(n=50, d=20, NN=100)

Number of discrepency 0, percent discrepency 0.000000
Check that the aligment matrix is orthogonal
L1 norm of diff is 0.000000


In [6]:

import numpy.linalg as la
from numpy import bmat
from scipy.linalg import expm, expm_frechet
from ManNullRange.manifolds.tools import (
    vecah, unvecah, asym)



# reporting statistics. Reporting the tables counting iteration, function valuations, improvements over generating vector

In [7]:

def Edist(flg, X, Y):
     """ Euclidean distance. Useful to compare two
     elememt
     """
     YTX = Y.T@X
     d2 = 2*(np.sum(flg.lbd * flg.lbd) - np.trace(
         (YTX*flg.lbd[None, :])@(YTX.T*flg.lbd[None, :])))
     return np.sqrt(max(d2, 0))
 

def make_lbd(dvec):
    dd = dvec.shape[0]

    def coef(a, dd):
        if dd % 2 == 0:
            if a < dd // 2 - 1:
                return - (dd // 2) + a + 1
            else:
                return - (dd // 2) + a + 2
        else:
            if a < (dd-1)//2:
                return -(dd-1)//2 + a
            else:
                return -(dd-1)//2 + a + 1

    dsum = dvec[1:].cumsum()
    lbd = np.concatenate([np.ones(dvec[a+1])*coef(a, dd)
                          for a in range(dsum.shape[0])])
    return lbd
    

def report_table(d_array, NN=50):
    import numpy.linalg as la
    from numpy import bmat
    from scipy.linalg import expm, expm_frechet
    from ManNullRange.manifolds.tools import (
        vecah, unvecah, asym)
    
    alp = np.array([1, 1])
    
    dvec = np.array(d_array)
    # dvec = np.array([0, 20, 15, 5])
    # dvec = np.array([0, 10, 5, 5])    
    # dvec[0] = 1000 - dvec[1:].sum()
    # dvec = np.array([5, 3, 1])
    q = dvec.shape[0]-1
    alpha = randint(1, 10, (q, q+1)) * .1
    alpha[:, 0] = alp[0]
    alpha[:, 1:] = alp[1]
    m, d = dvec.sum(), dvec[1:].sum()
    man = RealFlag(dvec, alpha=alpha, log_stats=True)

    lbd = make_lbd(dvec)
    
    def llbd(mat):
        return lbd[:, None]*mat

    def rlbd(mat):
        return mat*lbd[None, :]    
    
    
    mean_sts = {}
    klist = ['niter', 'nfvals', 'success', 'dist', 'improve']
    fts = dict((kk, 1) for kk in klist)
    fts['success'] = 100    
    for radius in [.5, .99, 1.3]:
        np.random.seed(0)
        rkey = 'rd%0.2f' % radius
        mean_sts[rkey] = {}
        for alx in np.array([1, 5, 10, 12]):
            tots = dict((k, {0: 0, 1: 0}) for k in klist)
            X = man.rand()        
            # np.random.seed(0)            
            man.alpha[:, 1:] = alx*.1
            print('Doing alpha=%.2f' % man.alpha[0, 1])
            cnt = 0
            all_sts = {}
            itr = {0: 0}
            nhev = {0: 0}
            for i in range(NN):
                xi = man.randvec(X)
                X1 = man.exp(X, radius*np.pi*xi)
                try:
                    #lgs0, sts0 = man.log(X, X1, init_type=0)
                    lgs0, iter0, fvals, fjacs, done0 = log_steep_descent(man, X, X1)
                    # lgs0, iter0, fvals, fjacs, done0 = log_descent(man, X, X1)
                    lgs1, sts1 = log_lbfgs(man, X, X1)
                except Exception as ex:
                    print(ex)
                    sts0 = [('success', False)]
                    lgs0 = np.zeros_like(X)
                # msg0 = dict(sts0)
                Norm0 = Edist(man, man.exp(X, lgs0), X1)
                Norm1 = Edist(man, man.exp(X, lgs1), X1)
                msg0 = {'success': done0 or Norm0 < 1e-3, 'nfvals': fvals + fjacs*3, 'niter': iter0}
                msg1 = {'success': sts1['firstorderopt'] < 1e-3, 'nfvals': sts1['funcCount']*3, 'niter': sts1['iterations']}
                all_sts[i] = {0: sts1}
                v0 = man.norm(X, lgs0)
                v1 = man.norm(X, lgs1)
                # diff = (lbd*lbd).sum() - np.trace(rlbd((X1.T@X1a))@rlbd((X1a.T@X1)))
                diff = Norm1
                if diff > 1e-3:
                    msg1['success'] = False
                    # if not msg1['success']:                     
                    # pdb.set_trace()
                    # pass
                if msg0['success'] and (v0 <= radius*np.pi + 1e-3):
                      tots['dist'][0] += 1
                      tots['improve'][0] += 1-man.norm(X, lgs0)/radius/np.pi
                if msg1['success'] and (v1 <= radius*np.pi + 1e-3):
                      tots['dist'][1] += 1
                      tots['improve'][1] += 1-man.norm(X, lgs1)/radius/np.pi
                                        
                for kk in klist:
                    if kk in msg0:
                        tots[kk][0] += msg0[kk]
                    if kk in msg1:
                        tots[kk][1] += msg1[kk]

            mean_sts[rkey][alx] = dict((kk, [tots[kk][0]/NN*fts[kk],
                                             tots[kk][1]/NN*fts[kk]])
                                       for kk in klist)
            print('radius %s alpha %f' % (rkey, man.alpha[0, 1]))
    # print(mean_sts)
    fname = 'tbl_flag_r_al_m%d_d%d.p' % (m, d)
    np.savez_compressed(fname, mean_sts)
    return mean_sts


In [8]:
mean_sts = report_table([960, 20, 15, 5], 50)
# mean_sts = report_table([50, 2, 3, 4], NN=50)
# mean_sts = report_table([2, 3, 4], NN=50)
# %pdb
mean_sts


Doing alpha=0.10
radius rd0.50 alpha 0.100000
Doing alpha=0.50
radius rd0.50 alpha 0.500000
Doing alpha=1.00
radius rd0.50 alpha 1.000000
Doing alpha=1.20
radius rd0.50 alpha 1.200000
Doing alpha=0.10
radius rd0.99 alpha 0.100000
Doing alpha=0.50
radius rd0.99 alpha 0.500000
Doing alpha=1.00
radius rd0.99 alpha 1.000000
Doing alpha=1.20
radius rd0.99 alpha 1.200000
Doing alpha=0.10
radius rd1.30 alpha 0.100000
Doing alpha=0.50
radius rd1.30 alpha 0.500000
Doing alpha=1.00
radius rd1.30 alpha 1.000000
Doing alpha=1.20
radius rd1.30 alpha 1.200000


{'rd0.50': {1: {'dist': [1.0, 1.0],
   'improve': [-1.1336944605311317e-08, 1.0392466969211255e-08],
   'nfvals': [74.8, 39.0],
   'niter': [18.48, 11.9],
   'success': [100.0, 100.0]},
  5: {'dist': [1.0, 1.0],
   'improve': [-2.840333219289448e-07, 2.6996864699313506e-09],
   'nfvals': [60.3, 33.36],
   'niter': [15.04, 10.02],
   'success': [100.0, 100.0]},
  10: {'dist': [1.0, 1.0],
   'improve': [-9.740249197776052e-07, 4.529120558594002e-09],
   'nfvals': [96.02, 42.3],
   'niter': [23.78, 12.82],
   'success': [100.0, 100.0]},
  12: {'dist': [1.0, 1.0],
   'improve': [-1.1830715369587352e-06, -1.076711382275164e-09],
   'nfvals': [200.64, 48.78],
   'niter': [49.86, 13.26],
   'success': [100.0, 100.0]}},
 'rd0.99': {1: {'dist': [0.34, 1.0],
   'improve': [-1.746099665966838e-08, 2.216715726532925e-09],
   'nfvals': [382.0, 72.96],
   'niter': [93.88, 18.94],
   'success': [34.0, 100.0]},
  5: {'dist': [1.0, 1.0],
   'improve': [-4.294051325284265e-06, -4.659157357700394e-09],
 

In [9]:
mean_sts = report_table([5, 3, 1], 50)
mean_sts
# import pdb
# %pdb

Doing alpha=0.10
radius rd0.50 alpha 0.100000
Doing alpha=0.50
radius rd0.50 alpha 0.500000
Doing alpha=1.00
radius rd0.50 alpha 1.000000
Doing alpha=1.20
radius rd0.50 alpha 1.200000
Doing alpha=0.10
radius rd0.99 alpha 0.100000
Doing alpha=0.50
radius rd0.99 alpha 0.500000
Doing alpha=1.00
radius rd0.99 alpha 1.000000
Doing alpha=1.20
radius rd0.99 alpha 1.200000
Doing alpha=0.10
radius rd1.30 alpha 0.100000
Doing alpha=0.50
radius rd1.30 alpha 0.500000
Doing alpha=1.00
radius rd1.30 alpha 1.000000
Doing alpha=1.20
radius rd1.30 alpha 1.200000


{'rd0.50': {1: {'dist': [0.84, 1.0],
   'improve': [-0.00010457678072099519, 5.169906880375663e-10],
   'nfvals': [481.68, 111.9],
   'niter': [99.14, 31.8],
   'success': [88.0, 100.0]},
  5: {'dist': [0.2, 1.0],
   'improve': [-5.289342190701429e-05, -3.534762036405681e-09],
   'nfvals': [335.96, 78.18],
   'niter': [83.18, 22.12],
   'success': [84.0, 100.0]},
  10: {'dist': [0.3, 1.0],
   'improve': [-9.480612603873784e-05, -1.190391103467192e-09],
   'nfvals': [219.7, 57.3],
   'niter': [52.76, 16.28],
   'success': [100.0, 100.0]},
  12: {'dist': [0.34, 1.0],
   'improve': [-0.00011149223325118473, -1.4576884810324487e-10],
   'nfvals': [318.86, 55.2],
   'niter': [70.3, 15.5],
   'success': [88.0, 100.0]}},
 'rd0.99': {1: {'dist': [0.32, 1.0],
   'improve': [0.12794355724336512, 0.31130780697023036],
   'nfvals': [504.58, 261.18],
   'niter': [100.0, 73.86],
   'success': [32.0, 100.0]},
  5: {'dist': [0.82, 0.96],
   'improve': [0.2072897641367045, 0.2593536946954417],
   'nfva

In [10]:
def _print_tbl(mean_sts):
    header = [r'dist ($\pi$)', r'$\alpha$', r'\#Iteration', r'\#Evals',
              r'\%Succ', '\\%Improve', '\\%Not worse']
    radius = sorted(mean_sts)

    def print_block():
        for r in radius:
            for alf in sorted(mean_sts[r].keys()):
                ddc = mean_sts[r][alf]
                out_str = '%s & %0.2f & %0.2f & %.2f &  %.f & %.f & %.f\\\\'
                print(
                    out_str % (
                        r[2:], alf / 10, ddc['niter'][0],
                        ddc['nfvals'][0],
                        ddc['success'][0],
                        np.abs(ddc['improve'][0]*100),
                        ddc['dist'][0]*100))
    print('&'.join(header) + r'\\')
    print_block()

def print_tbl(mean_sts):
    header = [r'dist ($\pi$)', r'$\alpha$', r'\#Iteration', r'\#Calc',
              r'\%Succ', '\\%Improve', '\\%Not worse distance']
    radius = sorted(mean_sts)

    def print_block():
        for r in radius:
            for alf in sorted(mean_sts[r].keys()):
                ddc = mean_sts[r][alf]
                out_str = '%s & %0.2f & %0.2f/%0.2f & %.2f/%.2f &  %.f/%.f & %.f/%.f & %.f/%.f\\\\'
                print(
                    out_str % (
                        r[2:], alf / 10, ddc['niter'][0], ddc['niter'][1],
                        ddc['nfvals'][0],
                        ddc['nfvals'][1],
                        ddc['success'][0], ddc['success'][1],
                        np.abs(ddc['improve'][0]*100), np.abs(ddc['improve'][1]*100),
                        ddc['dist'][0]*100, ddc['dist'][1]*100))
    print('&'.join(header) + r'\\')
    print_block()


def print_all():
    m1000_40 = np.load('tbl_flag_r_al_m1000_d40.p.npz', allow_pickle=True)['arr_0'].tolist()
    print_tbl(m1000_40)

    m4_2 = np.load('tbl_flag_r_al_m9_d4.p.npz', allow_pickle=True)['arr_0'].tolist()
    print_tbl(m4_2)

print_all()

dist ($\pi$)&$\alpha$&\#Iteration&\#Calc&\%Succ&\%Improve&\%Not worse distance\\
0.50 & 0.10 & 18.48/11.90 & 74.80/39.00 &  100/100 & 0/0 & 100/100\\
0.50 & 0.50 & 15.04/10.02 & 60.30/33.36 &  100/100 & 0/0 & 100/100\\
0.50 & 1.00 & 23.78/12.82 & 96.02/42.30 &  100/100 & 0/0 & 100/100\\
0.50 & 1.20 & 49.86/13.26 & 200.64/48.78 &  100/100 & 0/0 & 100/100\\
0.99 & 0.10 & 93.88/18.94 & 382.00/72.96 &  34/100 & 0/0 & 34/100\\
0.99 & 0.50 & 73.26/18.66 & 293.90/66.96 &  100/100 & 0/0 & 100/100\\
0.99 & 1.00 & 87.06/18.46 & 439.38/75.78 &  64/100 & 0/0 & 64/100\\
0.99 & 1.20 & 66.62/24.56 & 429.00/89.22 &  100/100 & 0/0 & 100/100\\
1.30 & 0.10 & 85.46/31.20 & 425.88/106.50 &  100/100 & 0/0 & 100/100\\
1.30 & 0.50 & 47.08/25.70 & 226.60/88.86 &  100/100 & 0/0 & 100/100\\
1.30 & 1.00 & 75.18/28.48 & 477.80/106.86 &  100/100 & 0/0 & 100/100\\
1.30 & 1.20 & 100.00/38.24 & 759.38/130.92 &  100/100 & 0/0 & 100/100\\
dist ($\pi$)&$\alpha$&\#Iteration&\#Calc&\%Succ&\%Improve&\%Not worse distance\\
0

In [11]:
from IPython.display import HTML, display 

def print_html(mean_sts):
    header = [r'dist ($\pi$)', r'$\alpha$', r'#Iteration', r'#Calc',
              r'%Succ', '%Improve', '%Not worse distance']
    radius = sorted(mean_sts)
    
    def print_block():
        ret = ''
        for r in radius:
            for alf in sorted(mean_sts[r].keys()):
                ddc = mean_sts[r][alf]
                out_str = '<tr><td>%s </td><td> %0.2f </td><td> %0.2f/%0.2f </td><td> %.2f/%.2f </td><td>  %.f/%.f </td><td> %.f/%.f </td><td> %.f/%.f</td></tr>'
                ret = ret + \
                    out_str % (
                        r[2:], alf / 10, ddc['niter'][0], ddc['niter'][1],
                        ddc['nfvals'][0],
                        ddc['nfvals'][1],
                        ddc['success'][0], ddc['success'][1],
                        np.abs(ddc['improve'][0]*100), np.abs(ddc['improve'][1]*100),
                        ddc['dist'][0]*100, ddc['dist'][1]*100)
        return ret
    html = '<table border="1">'
    html += "<tr><td> %s <td></tr>" % "</td><td>".join(header)                    
    html += print_block()
    html += '</table>'
    display(HTML(html))

def print_all_html():
    fname = 'tbl_flag_r_al_m1000_d40.p.npz'
    m1000_40 = np.load(fname, allow_pickle=True)['arr_0'].tolist()
    print("Doing Flag [960, 20, 15, 5] %s" % fname)
    print_html(m1000_40)

    fname = 'tbl_flag_r_al_m9_d4.p.npz'
    print("Doing Flag [5, 3, 1] %s" % fname)
    m4_2 = np.load(fname, allow_pickle=True)['arr_0'].tolist()
    print_html(m4_2)

print_all_html()    
    


Doing Flag [960, 20, 15, 5] tbl_flag_r_al_m1000_d40.p.npz


Doing Flag [5, 3, 1] tbl_flag_r_al_m9_d4.p.npz


Estimate execution time for expm

In [12]:
from scipy.linalg import expm
NS = 1000
A = np.random.randn(NS, NS)
A = A -A.T
A = A/np.linalg.norm(A)
%timeit -n 5 expm(A)

5 loops, best of 5: 528 ms per loop


# Riemannian Center of Mass

$\newcommand{\R}{\mathbf{R}}$
$\newcommand{\rM}{\mathrm{M}}$
$\newcommand{\diag}{\mathrm{diag}}$
$\newcommand{\sfg}{\mathsf{g}}$
$\newcommand{\Log}{\mathrm{Log}}$
## First, data generation from the tangent normal model
Similar to the Stiefel case, we consider the flag case.
Consider a vector $[d_0, d_1,\cdots, d_q]$ of positive integers, with $n = d_0 + d_1+\cdots + d_q$, $d =d_1,\cdots, d_q$.
Set $\dim_A = d(d-1)/2- \sum_{i=1}^q d_i(d_i-1)/2$, $\dim_B = m(m-d)$
The tangent space has dimension $\dim_A + \dim_B$. At point $Y\in \rM$ ($\rM$ is the manifold), consider the complementary basis $Y_{\perp}$ so that $(Y, Y_{\perp})$ is of determinant $1$ and orthogonal, consider the basis $YA_i + Y_{\perp}B_i$, $0\leq i < \dim M$, Here:
* for $0 \leq i < \dim_A$, $B_i = 0$ while $A_i$'s forms a basis of the space of antisymmetric matrices of size $d\times d$ and zero diagonal blocks of the form $2^{-\frac{1}{2}}(e_{ab} - e_{ba})$, where $e_{ab}$ is an elementary matrix of $\R^{d\times d}$ which is zero everywhere except for entry of position $a,b)$. 
* for $\dim_A \leq i < \dim_A + \dim_B$, $A_i = 0$ while $B_i$ is the standard
basis of $\R^{(m-d)\times d}$ of the form $E_{ab}$, where $E_{ab}$ are elementary matrices with only entry $ab$ equal $1$, and other entries zero, .

We pick random vectors $\{v_{j}\in \R^{\dim_A + \dim_B}| 0\leq j N\}$ following the normal distribution $N(0, \diag(\alpha_1^{-1}I_{\dim_A}, \alpha_0^{-1}I_{\dim_B}))$, write $v_j = (v_{ji} )_{0<i \leq \dim_A + \dim_B}$, then
$$\eta_j = \sum v_{ji}(YA_i + Y_{\perp}B_i)$$
are tangent vectors at $Y$, following a normal distribution defined by the metric $\sfg$. If the geodesic ball of this metric is big enough, covers most events in this statistic, the exponential map defines points $Q_j\in\rM$, which follows what we will call a tangent-normal distribution. Points outside the geodesic ball on $\rM$ are assigned a residual measure that will be negligible. 

The following function implements this data generation algorithm. To enforce the condition that $\eta_i$ lies within a geodesic ball where the logarithm map is easily compute, we need $\alpha_0$ scaled appropriately.

 For convenience, we use the normalized manifold with parameters $(1, \alpha:=\alpha_1/\alpha_0)$. The square norm of $\eta_i$ follows the $\chi^2(\dim_{\rM})$ distribution approximately, which is well known to have mean $\dim_{\rM}$ and standard deviation $\sqrt{2\dim_{\rM}}$. The ratio between the standard deviation and the mean is $\sqrt{2/\dim_{\rM}}$. At $m = 1000, d= 50$ this ratio is $0.64\%$, thus the length of the norm is very concentratedly distributed. Thus if we want the mean distance under the normalized metric (with $\alpha_0=1$) to be within $c\pi$, we scale by
$c\pi(\dim_B + \dim_A)^{-1/2}$

The data-generation parameters are thus $\alpha_0 = (\dim_B + \dim_A)/(c\pi)^2$, $\alpha_1 = \alpha\alpha_0$, which would generate data with mean norm (under the normalized metric) is $c\pi$.

In [13]:
from scipy.linalg import null_space
def randQ(flg, Y, N, npi):
  m, d = flg.dvec.sum(), flg.dvec[1:].sum()
  alpha = flg.alpha[0, :2]
  dimB = flg.dvec[0]*d
  dimA = flg.dim - dimB
  
  dimAb = dimA + dimB

  # this is the effective alpha
  # alphax = np.array([dimB + dimA/alf, alf*dimB + dimA])/(np.pi*npi)/(np.pi*npi)

  vmat = np.random.randn((dimAb * N)).reshape(dimAb, N)
  vmat[:dimA, :] = vmat[:dimA, :]*np.sqrt(alpha[1])
  vmat[dimA:, :] = vmat[dimA:, :]*np.sqrt(alpha[0])
    
  Yperp = null_space(Y.T)
  Q = np.empty((m, d, N))
  for i in range(N):
    A, B = unvecAR(flg, vmat[:, i])
    etai = Y@A + Yperp@B
    # print(flg.norm(Y, etai))
    sclf = np.pi*npi/np.sqrt(dimB + dimA)
    Q[:, :, i] = flg.exp(Y, etai*sclf)
  return Q, sclf



In [14]:
lg2pi = np.log(2*np.pi)

from scipy.linalg import null_space
sqrt2 = np.sqrt(2.)

def unvecAR(flg, avec):
    d = flg.dvec[1:].sum()
    m = flg.dvec.sum()
    p = flg.dvec.shape[0]-1
    A = np.zeros((d, d))
    R = np.zeros((m-d, d))
    gdc = flg._g_idx
    be = 0
    for r in range(1, p+1):
          if r not in gdc:
              continue
          br, er = gdc[r]
          for s in range(r+1, p+1):
              if s <= r:
                  continue
              bs, es = gdc[s]
              dr = er - br
              ds = es - bs
              A[br:er, bs:es] = (avec[be: be+dr*ds]/sqrt2).reshape(dr, ds)
              A[bs:es, br:er] = - A[br:er, bs:es].T
              be += dr*ds
    R = avec[be:].reshape(m-d, d)
    return A, R



def riemannian_center_of_mass_loop(dvec, alf, N, radius=.5, dump=False):
    # now do Riemannian center of mass
    import time    
    np.random.seed(0)    
    
    # dvec = np.array([0, 30, 15, 5])
    # dvec = np.array([0, 10, 5, 5])
    
    # dvec[0] = 1000 - dvec[1:].sum()
    p = dvec.shape[0]-1
    alpha = np.full((p, p+1), alf)
    alpha[:, 0] = 1
    # alpha[:, 0] = 1.
    
    # N = 50
    m, d = dvec.sum(), dvec[1:].sum()
    man = RealFlag(
        dvec, alpha=alpha, log_stats=False, log_method='trust-krylov')
    # man.log_gtol = 1e-6

    # npi = .8
    Y = man.rand()
    Q, sclf = randQ(man, Y, N, radius)
    if False:
      Q = np.zeros((m, d, N))    
      fts = np.zeros(N)

      for i in range(N):
          fts[i] = np.random.uniform(radius*.8, radius)*np.pi
          Q[:, :, i] = man.exp(Y, fts[i]*man.randvec(Y))
    print('sclf=%f' % sclf)
    print('generating alpha0=%f' % (1/sclf/sclf))
    man.log_stats = True
    tot_stats_all = {}
    init_type = 0
    for alx in [1, 5, 10]:
        start_time = time.time()        
        man.alpha[:, 1:] = alx*.1
        man.log_stats = True
        if alx != 10:
            ctr, nllk, tot_stats, itr_stats = RCM(man, Q, 0, init_type=init_type)
        else:
            ctr, nllk, tot_stats, itr_stats = RCM(man, Q, 0, init_type=init_type)
        print('dvec=%s alpha_1=%.f radius=%.f dist_to_y=%.f nllk=%.f bt=%.f' %
              (str(man.dvec), man.alpha[0, 1], radius,
               man.norm(Y, man.log(Y, ctr)[0]), nllk, tot_stats['bt']))
        for itr in sorted(itr_stats.keys()):
            print('itr=%d stat=%s' % (itr, itr_stats[itr]))
        print(tot_stats)
        tot_stats_all[alx] = (tot_stats, itr_stats)
        end_time = time.time()
        # print(start_time, end_time, end_time - start_time)
        print('time elapsed %f' % (end_time - start_time))
    if dump:
      import pickle
      with open('rcm_flag.p', 'wb') as ff:
          pickle.dump(tot_stats_all, ff)
    return tot_stats_all
        

def RCM(man, Q, init_idx=None, init_type=0):
    m, d = man.dvec.sum(), man.dvec[1:].sum()
    N = Q.shape[2]
    n_iter = 100
    a, _, b = np.linalg.svd(0.8*Q[:, :, 0] + 0.2*Q.mean(axis=2), full_matrices=False)
    if init_idx is None:
        Yi = a@b
    else:
        Yi = Q[:, :, init_idx]
    Yinew = Yi.copy()
    k_list = ['success', 'nfvals', 'niter']
    tot_stats = dict((kk, 0) for kk in k_list)
    tot_stats['cnt'] = 0
    itr_mean = {}
    tot_mean = {}

    for itr in range(n_iter):
        logs = np.zeros((m, d))
        itr_stats = dict((kk, 0) for kk in k_list)
        itr_stats['cnt'] = 0
        itr_mean[itr] = {}
        variance = 0        
        nllkx = 0
        for i in range(N):
            itr_stats['cnt'] += 1
            tot_stats['cnt'] += 1
            if not man.log_stats:
                ret, _ = log_lbfgs(man, Yi, Q[:, :, i])
            else:
                ret, sts = log_lbfgs(man, Yi, Q[:, :, i])
                msg = dict(sts)
                msg['success'] = Edist(man, man.exp(Yi, ret), Q[:, :, i])
                if msg['success']:
                  itr_stats['success'] += 1
                  tot_stats['success'] += 1

                itr_stats['niter'] += msg['iterations']
                tot_stats['niter'] += msg['iterations']                        

                itr_stats['nfvals'] += msg['funcCount']*3
                tot_stats['nfvals'] += msg['funcCount']*3                        

                if False and (msg['success'] is not True):
                    print('Not success itr = %d i=%d %s' % (itr, i, str(msg)))
            jlogs = man.J(Yi, ret)
            if np.isnan(ret).sum() == 0:
                logs += 1/N*ret
                variance += 1/N*man.inner(Yi, ret, ret)
                nllkx += 0.5/N*man.base_inner_ambient(ret, man.g_inv(Yi, ret))
        nlg = man.norm(Yi, logs)
        for kk in k_list:
            if kk == 'success':
                itr_mean[itr][kk] = itr_stats[kk] / itr_stats['cnt']
            else:
                itr_mean[itr][kk] = itr_stats[kk] / itr_stats['success']

        # sometime the optimizer return sucess = False
        # but opt value is still good to use
        # so count of success does not mean that much
        # iteration get much slower
        itr_mean[itr]['success'] *= 100
        itr_mean[itr]['norm_RCM_gradient'] = nlg
        itr_mean[itr]['RCM_variance'] = variance
        # print('itr=%d dis=%f %s' % (itr, nlg, str(itr_mean)))
        if nlg < 1e-5:
            break
        Yinew = man.exp(Yi, logs)
        Yi = Yinew
        for kk in k_list:
            if kk == 'success':
                tot_mean[kk] = (tot_stats[kk] / tot_stats['cnt'])
            else:
                tot_mean[kk] = (tot_stats[kk] / tot_stats['success'])
        tot_mean['success'] *= 100
    dimB = man.dvec[0]*d
    dimA = man.dim - dimB
    # print(man.alpha[0, :2], dimA, dimB, man.dim)
    # print(man.dim/2*lg2pi, 0.5*dimB*np.log(man.alpha[0, 0]) + dimA*np.log(man.alpha[0, 1]), nllkx)
    ibt = nllkx/man.dim*2
    nllk = man.dim/2*lg2pi +  0.5*(dimB*np.log(man.alpha[0, 0]) + dimA*np.log(man.alpha[0, 1]))
    nllk += man.dim/2*np.log(ibt) + nllkx/ibt
    tot_mean['nllk'] = nllk
    tot_mean['bt'] = 1/ibt

    return Yi, nllk, tot_mean, itr_mean


In [15]:
dvec = np.array([2, 3, 3, 7])

tot_stats_all = riemannian_center_of_mass_loop(dvec, 0.9, 50, radius=.5)
print(tot_stats_all)

sclf=0.179009
generating alpha0=31.206925
dvec=[2 3 3 7] alpha_1=0 radius=0 dist_to_y=0 nllk=-13 bt=5
itr=0 stat={'success': 98.0, 'nfvals': 146.57142857142858, 'niter': 42.775510204081634, 'norm_RCM_gradient': 0.9277237043324384, 'RCM_variance': 1.9344681737692524}
itr=1 stat={'success': 100.0, 'nfvals': 93.36, 'niter': 27.16, 'norm_RCM_gradient': 0.12448279414464836, 'RCM_variance': 0.9552191714625777}
itr=2 stat={'success': 100.0, 'nfvals': 93.78, 'niter': 27.02, 'norm_RCM_gradient': 0.015659762725192067, 'RCM_variance': 0.9377885999319386}
itr=3 stat={'success': 100.0, 'nfvals': 91.2, 'niter': 26.48, 'norm_RCM_gradient': 0.002021264867158885, 'RCM_variance': 0.9375119112847161}
itr=4 stat={'success': 100.0, 'nfvals': 91.62, 'niter': 26.4, 'norm_RCM_gradient': 0.000267172951506419, 'RCM_variance': 0.9375072877066023}
itr=5 stat={'success': 100.0, 'nfvals': 92.04, 'niter': 26.6, 'norm_RCM_gradient': 3.61368426999113e-05, 'RCM_variance': 0.9375072042660354}
itr=6 stat={'success': 100.

In [16]:
dvec = np.array([0, 30, 15, 5])
# dvec = np.array([0, 10, 5, 5])
dvec[0] = 1000 - dvec[1:].sum()

tot_stats_all = riemannian_center_of_mass_loop(dvec, 0.9, 50, radius=.5, dump=True)
print(tot_stats_all)

sclf=0.007157
generating alpha0=19524.592088
dvec=[950  30  15   5] alpha_1=0 radius=0 dist_to_y=0 nllk=-168274 bt=17882
itr=0 stat={'success': 92.0, 'nfvals': 49.56521739130435, 'niter': 12.869565217391305, 'norm_RCM_gradient': 1.5150318311848519, 'RCM_variance': 4.719847010153678}
itr=1 stat={'success': 76.0, 'nfvals': 51.31578947368421, 'niter': 14.342105263157896, 'norm_RCM_gradient': 0.025043808295191258, 'RCM_variance': 2.386546024110089}
itr=2 stat={'success': 80.0, 'nfvals': 48.75, 'niter': 13.6, 'norm_RCM_gradient': 0.0004153229836627649, 'RCM_variance': 2.3859084613961237}
itr=3 stat={'success': 78.0, 'nfvals': 50.0, 'niter': 14.0, 'norm_RCM_gradient': 6.97323477536459e-06, 'RCM_variance': 2.3859082860142373}
{'success': 82.66666666666667, 'nfvals': 49.83870967741935, 'niter': 13.556451612903226, 'nllk': -168274.25497814108, 'bt': 17882.39951237058}
time elapsed 65.156249
dvec=[950  30  15   5] alpha_1=0 radius=0 dist_to_y=0 nllk=-170074 bt=19709
itr=0 stat={'success': 54.0, 

In [17]:
if True:
  import pickle
  with open("rcm_flag.p", 'rb') as pf:
    tbl = pickle.load(pf)
print("iter & 0.10 & 0.50 & 1.00")

# output to latex
for i in range(4):
  print("%d & %.3f & %0.3f & %0.3f \\\\" % (
      i,
      np.log10(tbl[1][1][i]['norm_RCM_gradient']),
      np.log10(tbl[5][1][i]['norm_RCM_gradient']),
      np.log10(tbl[10][1][i]['norm_RCM_gradient'])
      ))
print(" & %.f & %.f & %.f \\\\" %tuple ([tbl[i][0]['nllk'] for i in [1, 5, 10]]))
print(" & %.f & %.f & %.f \\\\" %tuple ([tbl[i][0]['bt'] for i in [1, 5, 10]]))

# formated output
from IPython.display import HTML, display 
html = '<table border="1">'
html += "<tr><td> itr</td> <td>0.1 </td> <td>0.5 </td> <td> 1. <td></tr>" 
for i in range(4):
  html += "<tr><td>%d </td> <td> %.3f </td><td> %0.3f </td><td> %0.3f </td></tr>" % (
      i,
      np.log10(tbl[1][1][i]['norm_RCM_gradient']),
      np.log10(tbl[5][1][i]['norm_RCM_gradient']),
      np.log10(tbl[10][1][i]['norm_RCM_gradient'])
      )
html += "<tr><td> NLLK  </td><td> %.f  </td><td> %.f </td><td> %.f </td></tr>" %tuple ([tbl[i][0]['nllk'] for i in [1, 5, 10]])
html += "<tr><td> alpha0  </td><td> %.f </td><td> %.f </td><td> %.f </td></tr>" %tuple ([tbl[i][0]['bt'] for i in [1, 5, 10]])
html + "</table>" 
display(HTML(html))




iter & 0.10 & 0.50 & 1.00
0 & 0.180 & 0.182 & 0.183 \\
1 & -1.601 & -1.607 & -1.610 \\
2 & -3.382 & -3.395 & -3.403 \\
3 & -5.157 & -5.179 & -5.192 \\
 & -168274 & -170074 & -170146 \\
 & 17882 & 19709 & 19961 \\


In [ ]:
grid = np.array([0.1, .5, 1.])

def runLlkRCM(n, dvec1, alf):
  print("######")
  p = dvec1.sum()
  dvec = np.concatenate([np.array([n-p]), dvec1])

  print("DOING n=%d dvec=%s alf=%.02f" % (n, str(dvec), alf))
  print("In the grid, %.02f should have the smallest negative log likelihood" % grid[np.argmin(np.abs(grid - alf))])
  riemannian_center_of_mass_loop(dvec, alf, 50, radius=0.5)

runLlkRCM(n=200, dvec1=np.array([10, 20, 5, 15]), alf=.9)
runLlkRCM(n=200, dvec1=np.array([10, 20, 5, 15]), alf=.6)
runLlkRCM(n=200, dvec1=np.array([10, 20, 5, 15]), alf=.2)
runLlkRCM(n=100, dvec1=np.array([10, 20, 5, 15]), alf=.15)
runLlkRCM(n=100, dvec1=np.array([10, 20, 5, 15]), alf=.45)
runLlkRCM(n=100, dvec1=np.array([10, 20, 5, 15]), alf=1.1)



######
DOING n=200 dvec=[150  10  20   5  15] alf=0.90
In the grid, 1.00 should have the smallest negative log likelihood
sclf=0.017164
generating alpha0=3394.259652
dvec=[150  10  20   5  15] alpha_1=0 radius=0 dist_to_y=0 nllk=-20670 bt=1869
itr=0 stat={'success': 98.0, 'nfvals': 61.89795918367347, 'niter': 18.102040816326532, 'norm_RCM_gradient': 1.4515670265924983, 'RCM_variance': 4.330270305005047}
itr=1 stat={'success': 60.0, 'nfvals': 79.1, 'niter': 20.0, 'norm_RCM_gradient': 0.02722200144284907, 'RCM_variance': 2.184003978317742}
itr=2 stat={'success': 46.0, 'nfvals': 102.52173913043478, 'niter': 26.08695652173913, 'norm_RCM_gradient': 0.0005283574570467805, 'RCM_variance': 2.1832487164102736}
itr=3 stat={'success': 48.0, 'nfvals': 97.25, 'niter': 24.958333333333332, 'norm_RCM_gradient': 1.0696382346561396e-05, 'RCM_variance': 2.1832484323458337}
itr=4 stat={'success': 28.000000000000004, 'nfvals': 167.14285714285714, 'niter': 42.857142857142854, 'norm_RCM_gradient': 2.24444035

# Print Hessian Spectra

In [ ]:
def test_hessian_at_zero_A_R():
    import numpy.linalg as la
    from numpy import bmat
    from scipy.linalg import expm, expm_frechet
    from ManNullRange.manifolds.tools import (
        vecah, unvecah, asym)
    np.random.seed(0)    
    alp = np.zeros(2)
    alp[0] = 1
    alp[1] = .8

    dvec = np.array([0, 30, 15, 5])
    dvec[0] = 1000 - dvec[1:].sum()    
    dvec = np.array([15, 3, 2, 2])

    def make_lbd():
        dd = dvec.shape[0]

        def coef(a, dd):
            if dd % 2 == 0:
                if a < dd // 2 - 1:
                    return - (dd // 2) + a + 1
                else:
                    return - (dd // 2) + a + 2
            else:
                if a < (dd-1)//2:
                    return -(dd-1)//2 + a
                else:
                    return -(dd-1)//2 + a + 1
                
        dsum = dvec[1:].cumsum()
        lbd = np.concatenate([np.ones(dvec[a+1])*coef(a, dd)
                              for a in range(dsum.shape[0])])
        # return .5 + lbd / lbd.sum()
        return lbd
    
    lbd = make_lbd()
    
    p = dvec.shape[0]-1
    alpha = randint(1, 10, (p, p+1)) * .1
    alpha[:, 0] = alp[0]
    alpha[:, 1:] = alp[1]
    sqrt2 = np.sqrt(2)
    N = 50
    m, d = dvec.sum(), dvec[1:].sum()
    man = RealFlag(
        dvec, alpha=alpha, log_stats=False, log_method='trust-krylov')
    
    def vec(A, R):
        # for A, take all blocks [ij with i > j]
        lret = []
        for r in range(1, p+1):
            gdc = man._g_idx                
            if r not in gdc:
                continue
            br, er = gdc[r]
            for s in range(r+1, p+1):
                if s <= r:
                    continue
                bs, es = gdc[s]
                lret.append(A[br:er, bs:es].reshape(-1)*sqrt2)

        lret.append(R.reshape(-1))
        return np.concatenate(lret)

    def unvec(avec):
        A = np.zeros((d, d))
        R = np.zeros((k, d))
        gdc = man._g_idx
        be = 0
        for r in range(1, p+1):
            if r not in gdc:
                continue
            br, er = gdc[r]
            for s in range(r+1, p+1):
                if s <= r:
                    continue
                bs, es = gdc[s]
                dr = er - br
                ds = es - bs
                A[br:er, bs:es] = (avec[be: be+dr*ds]/sqrt2).reshape(dr, ds)
                A[bs:es, br:er] = - A[br:er, bs:es].T
                be += dr*ds
        R = avec[be:].reshape(k, d)
        return A, R

    def asym(mat):
        return 0.5*(mat - mat.T)

    def make_vec(xi):
        return vec(X.T@xi, Q.T@xi)

    def conv_to_tan(A, R):
        return X@A + Q@R
    
    def calc_hessian_spec(lgs, X1):        
        XQ = np.array(np.bmat([X, Q]))
        X2 = XQ.T@(X1*lbd[None, :])@X1.T@XQ
                
        def dist(v):
            #  = (dist0a(v) - d)*2
            alf = man.alpha[0, 1] / man.alpha[0, 0]
            A, R = unvec(v)
            x_mat = np.array(
                np.bmat([[2*alf*A, -R.T], [R, zeros((k, k))]]))
            exh = expm(x_mat)
            ex = expm((1-2*alf)*A)
            Mid = (ex*lbd[None, :])@ex.T
            return (- trace(X2@exh[:, :d]@Mid@exh[:, :d].T))

        def jac(v):
            alf = man.alpha[0, 1] / man.alpha[0, 0]            
            gdc = man._g_idx
            A, R = unvec(v)
            x_mat = np.array(
                np.bmat([[2*alf*A, -R.T], [R, zeros((k, k))]]))
            exh = expm(x_mat)
            ex = expm((1-2*alf)*A)

            blk = np.zeros_like(exh)
            blk[:d, :] = (ex*lbd[None, :])@ex.T@exh[:, :d].T
            blkA = (lbd[:, None]*ex.T)@exh[:, :d].T@X2@exh[:, :d]

            fexh = 2*expm_frechet(x_mat, blk@X2)[1]
            fex = 2*expm_frechet((1-2*alf)*A, blkA)[1]

            for r in range(1, p+1):
                if r not in gdc:
                    continue
                br, er = gdc[r]            
                fexh[br:br, br:br] = 0
                fex[br:br, br:br] = 0

            return vec(
                (1-2*alf)*asym(fex) + 2*alf*asym(fexh[:d, :d]),
                fexh[d:, :d] - fexh[:d, d:].T)
        
        def hessp(v, xi):
            dlt = 1e-8
            return (jac(v+dlt*xi) - jac(v))/dlt
        
        mat = np.zeros((tdim, tdim))
        Alg = X.T@lgs
        Rlgs = Q.T@lgs
        vlgs = vec(Alg, Rlgs)
        for i in range(tdim):
            av = np.zeros(tdim)
            av[i] = 1
            mat[:, i] = hessp(vlgs, av)
        mat = 0.5*(mat + mat.T)
        evl, evec = la.eigh(mat)
        return evl

    k = min(d, m-d)
    adim = (dvec[1:].sum()*dvec[1:].sum() - (dvec[1:]*dvec[1:]).sum()) // 2
    tdim = d*k + adim

    X = man.rand()
    eta0 = man.randvec(X)
    Q, _ = la.qr(eta0 - X@X.T@eta0)

    X1 = X.copy()
    # X1 = man.exp(X, 1*eta0)
    
    v = np.zeros(tdim)
    
    # A0, R0 = unvec(np.zeros((tdim)))
    # Q = make_Q(k)
    spt = calc_hessian_spec(np.zeros((m, d)), X1)
    print("Eigenvalues of the Hessian at 0")
    print(spt)

    from scipy.optimize import minimize_scalar
    man.alpha[:, 1:] = .01-.1
    cut_val = {}

    for aj in range(12):
        np.random.seed(0)
        man.alpha[:, 1:] += .1
        NN = 50
        cut_val[aj] = {}
        for ii in range(NN):
            xi = man.randvec(X)

            def make_Q(k):
                Q, _ = la.qr(xi - X@X.T@xi)
                return Q
            Q = make_Q(d)

            def min_spec(t):
                Y1 = man.exp(X, t*xi)
                spt = calc_hessian_spec(t*xi, Y1)
                # print(spt)
                return spt[0]*spt[0]

            ret = minimize_scalar(
                min_spec, method='bounded', bounds=(.01*np.pi, 1.4*np.pi))
            if ret['success'] and ret['fun'] < 1e-3:
                cut_val[aj][ii] = (ret['x'], ret['fun'])
            else:
                print(ret)
        print('quasi cut values for alpha=%f' % man.alpha[0, 1])
        print([cut_val[aj][a] for a in
               sorted(cut_val[aj], key=lambda i: cut_val[aj][i][0])])
    print("spetra - changing with t")
    if True:        
      for i in range(100):
          t = np.pi*1.4*i/100
          X1 = man.exp(X, t*xi)
          spt = sorted(calc_hessian_spec(t*xi, X1))
          print(t, spt[0], spt[-1])

test_hessian_at_zero_A_R()

Eigenvalues of the Hessian at 0
[1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 4. 4. 4. 4. 4. 4. 8. 8. 8.
 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 9. 9. 9. 9. 9. 9.]
quasi cut values for alpha=0.010000
[(2.7722804309874256, 7.078546025755227e-21), (2.917554094500036, 5.067394649519822e-17), (3.0022029492939266, 7.254302320885777e-18), (3.0110014141916244, 1.8366469889547734e-17), (3.0291646127768286, 2.3603356328767124e-17), (3.029854229364688, 4.578148891168654e-19), (3.031346423408614, 3.921047886633701e-18), (3.036372462965425, 3.245686609258509e-17), (3.0426251935744464, 6.700308467770912e-17), (3.0465096735547696, 1.5520860708864122e-16), (3.051101736145782, 2.077688589343592e-17), (3.051179355447505, 1.3572396822949648e-08), (3.0518926704289324, 1.5270416333506595e-16), (3.05971716435313, 3.035909524341139e-18), (3.0644517198021335, 2.3329398581444534e-18), (3.0726685735471455, 1.5242938667687229e-18), (3.0762695061